In [ ]:
import json
import matplotlib.pyplot as plt

In [ ]:
hpi_yellow = '#f6a800'
hpi_orange = '#dd6108'
hpi_red = '#b1063a'
hpi_gray = '#5a6065'
hpi_blue = '#007a9e'
hpi_green = '#007f6f'
hpi_lime = '#779a0b'
hpi_purple = '#7664a0'

In [ ]:
def load_file(name):
    with open(name, 'r') as f:
        data = json.loads(f.read())
    return data

def plot_times(series: list[list[float]], labels: list[str] = []):
    fig, ax = plt.subplots()
    fig.set_size_inches(5, len(series) / 2)
    parts = ax.violinplot(dataset=series, vert=False, showextrema=False, showmedians=True)
    ax.set_title("Client-side action processing times")
    ax.set_xlim(0)
    ax.set_xlabel('Per-action time in ms')
    ax.set_ylabel('Configuration')
    ax.set_yticks(ticks=range(1,len(series)+1), labels=labels)
    
    ax.grid(True)
    
    for pc in parts['bodies']:
        pc.set_alpha(1)
        pc.set_facecolor(hpi_orange)
        pc.set_edgecolor('k')
    parts['cmedians'].set_color(hpi_red)

    
    plt.show()

def load_series(fn):
    data = load_file(fn)
    plot_series = []
    plot_labels = []
    for session in data:
        settings = session['settings']
        for run in session['raw']:
            plot_series.append(run['times'])
            plot_labels.append(f'{settings["pas"]}P/{settings["ticks"]}T/{settings["run"]}R: {run["ctx"]["stage"]}')
    return plot_series, plot_labels

def plot_files(*names):
    s,l = [], []
    for i, f in enumerate(names):
        ds, dl = load_series(f)
        s += ds
        l += [f'{dll} ({i})' for dll in dl]
    plot_times(s, l)

In [ ]:
d = load_file('./../data/tick-patients/2023-06-08T14:25:57.929Z.json')
d += load_file('./../data/tick-patients/2023-06-08T14:41:29.940Z.json')
    
series = [run['raw']['[Exercise] Tick'] for run in d]
labels=[f'{run["settings"]["method"]}' for run in d]

plot_times(series, labels)

In [ ]:
plot_files(
    './../data/tick-patients/2023-06-12T09:42:50.885Z.json', 
    './../data/tick-patients/2023-06-12T09:55:08.684Z.json', 
    './../data/tick-patients/2023-06-12T10:07:25.733Z.json',
    './../data/tick-patients/2023-06-12T10:56:53.403Z.json',
           )

In [ ]:
class TimeStats:
    def __init__(self, raw_data):
        ctx = raw_data['ctx']
        self.type = ctx['type']
        self.stage = ctx['stage']
        stats = raw_data['stats']
        self.count = stats['count']
        self.total = stats['total']
        self.avg = stats['avg']
        self.stddev = stats['stddev']
        self.var = stats['variance']
        self.min = stats['min']
        self.med = stats['med']
        self.max = stats['max']

class TimeRaw:
    def __init__(self, raw_data):
        ctx = raw_data['ctx']
        self.type = ctx['type']
        self.stage = ctx['stage']
        self.times = raw_data['times']

class TimeSeries:
    def __init__(self, raw_data):
        settings = raw_data['settings']
        self.num_regions = settings['pas']
        self.num_loaded_regions = settings['loaded_pas']
        self.fraction_loaded = settings.get('fraction_pas', self.num_loaded_regions / self.num_regions if self.num_regions != 0 else 0)
        self.ticks_run = settings['ticks']
        self.iteration = settings['run']
        self.raw_results = [TimeRaw(x) for x in raw_data['raw']]
        self.stats_results = [TimeStats(x) for x in raw_data['stats']]
        

## Plots with partially loaded state

In [ ]:
def avg_time_per_loaded_fraction(raw_data, stage=None, action=None):
    out_series = {}
    raw_series = [TimeSeries(run) for run in raw_data]
    
    def is_selected(stats):
        return (stats.stage == stage or stage is None) and (action is None or action == stats.type)
    
    for run in raw_series:
        series = out_series.get(run.fraction_loaded, {})
        out_series[run.fraction_loaded] = series
        series[run.num_regions] = next(filter(is_selected, run.stats_results)).med
    
    return out_series

def plot_fraction_performance(series: dict):
    fig, ax = plt.subplots()
    ax.set_title("Client-side action processing times")
    
    ax.set_ylabel('Per-action time in ms')
    ax.set_xlabel('# of regions')
    ax.grid(True)
    for fraction, values in series.items():
        x, y = zip(*values.items())
        ax.plot(x, y, 'o', label=f'$p_r = {fraction}$')
    ax.legend()
    fig.show()

In [ ]:
raw_data = load_file('../data/partial-state-fractions/2023-07-17T10:05:08.246Z.json')
series = avg_time_per_loaded_fraction(raw_data)
plot_fraction_performance(series)